In [10]:

import findspark
from datetime import datetime
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, udf, col, when
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import os
from dotenv import load_dotenv

load_dotenv()



True

In [11]:
findspark.init()
topic_name = os.getenv('kafka_topic_name')

# spark_path = findspark.find()
# print(spark_path)
print(topic_name)



ticketmaster


In [ ]:
credentials_location = os.getenv('gcp_credentials_path')

# Spark Config
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('TwitterSentimentAnalysis') \
    .set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .set("spark.jars", "gs://path/to/spark-bigquery-latest.jar,gs://path/to/google-cloud-bigquery-latest.jar,/path/to/local-jar-file.jar") \
    .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)\
    .set("spark.jars", "gcs-connector-hadoop3-2.2.5.jar") 


sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

# Spark Context
# sc = spark.sparkContext
# sc.setLogLevel('ERROR')

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [13]:
schema = StructType([
    StructField("event_name", StringType()),
    StructField("event_type", StringType()),
    StructField("event_id", StringType()),
    StructField("event_url", StringType()),
    StructField("venue_name", StringType()),
    StructField("venue_id", StringType()),
    StructField("venue_zipcode", StringType()),
    StructField("venues_timezone", StringType()),
    StructField("venue_city", StringType()),
    StructField("venue_state_full", StringType()),
    StructField("venue_state_short", StringType()),
    StructField("venue_country_name", StringType()),
    StructField("venue_country_short", StringType()),
    StructField("venue_address", StringType()),
    StructField("venue_longitude", StringType()),
    StructField("venue_latitude", StringType()),
    StructField("attraction_name", StringType()),
    StructField("attraction_type", StringType()),
    StructField("attraction_id", StringType()),
    StructField("attraction_url", StringType()),
    StructField("attraction_segment_id", StringType()),
    StructField("attraction_segment_name", StringType()),
    StructField("attraction_genre_id", StringType()),
    StructField("attraction_genre_name", StringType()),
    StructField("attraction_subgenre_id", StringType()),
    StructField("attraction_subgenre_name", StringType()),
    StructField("event_start_date", StringType()),
    StructField("ticket_status", StringType()),
    StructField("event_start_time", StringType()),
    StructField("currency", StringType()),
    StructField("min_price", StringType()),
    StructField("max_price", StringType())
])

23/06/24 15:00:20 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-driver-0-1, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/24 15:00:20 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-driver-0-1, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-driver-0] Resetting offset for partition ticketmaster-0 to offset 4833.
23/06/24 15:00:20 INFO CheckpointFileManager: Writing atomically to gs://kafka-spark-data/spark-metadata/offsets/205 using temp file gs://kafka-spark-data/spark-metadata/offsets/.205.6d021bce-9f83-4973-bbdc-70b32df0eb66.tmp


In [14]:

PROJECT_ID = os.getenv('GCP_PROJECT_ID')
BUCKET = os.getenv('GCP_BUCKET')

dataset = os.getenv('GCP_dataset')
table = os.getenv('GCP_table')

gcs_metadata_folder = os.getenv('GCP_metadata_bucket')
gcs_data_folder = os.getenv('GCP_data_bucket')

print(PROJECT_ID)
print(BUCKET)
print(dataset)
print(gcs_data_folder)

23/06/24 15:00:21 INFO CheckpointFileManager: Renamed temp file gs://kafka-spark-data/spark-metadata/offsets/.205.6d021bce-9f83-4973-bbdc-70b32df0eb66.tmp to gs://kafka-spark-data/spark-metadata/offsets/205
23/06/24 15:00:21 INFO MicroBatchExecution: Committed offsets for batch 205. Metadata OffsetSeqMetadata(0,1687636820009,Map(spark.sql.streaming.stateStore.providerClass -> org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider, spark.sql.streaming.join.stateFormatVersion -> 2, spark.sql.streaming.stateStore.compression.codec -> lz4, spark.sql.streaming.stateStore.rocksdb.formatVersion -> 5, spark.sql.streaming.statefulOperator.useStrictDistribution -> true, spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion -> 2, spark.sql.streaming.multipleWatermarkPolicy -> min, spark.sql.streaming.aggregation.stateFormatVersion -> 2, spark.sql.shuffle.partitions -> 200))
global-maxim-338114
kafka-spark-data
global-maxim-338114.twitter_kafka_pyspark_test
gs://kafka-

23/06/24 15:00:22 INFO KafkaOffsetReaderConsumer: Partitions added: Map()
23/06/24 15:00:22 INFO KafkaOffsetReaderConsumer: Partitions added: Map()
23/06/24 15:00:22 INFO KafkaOffsetReaderConsumer: Partitions added: Map()
23/06/24 15:00:22 INFO KafkaOffsetReaderConsumer: Partitions added: Map()
23/06/24 15:00:23 INFO ParquetFileFormat: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
23/06/24 15:00:23 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/06/24 15:00:23 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
23/06/24 15:00:23 INFO SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
23/06/24 15:00:23 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/06/24 15:00:23 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporar

23/06/24 15:00:24 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor-2, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor] Seeking to EARLIEST offset of partition ticketmaster-0
23/06/24 15:00:24 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor-2, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 0.
23/06/24 15:00:24 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor-2, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor] Seeking to LATEST offset of partition ticketmaster-0
23/06/24 15:00:24 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881

23/06/24 15:00:26 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-1c839cdd-4a30-4906-99cf-fccc5e0b8cb4/_temporary/0/task_202306241500238432565655569741210_0000_m_000000/' directory.
23/06/24 15:00:27 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-1c839cdd-4a30-4906-99cf-fccc5e0b8cb4/' directory.
23/06/24 15:00:27 INFO BlockManagerInfo: Removed broadcast_0_piece0 on nics-mbp.attlocal.net:52028 in memory (size: 94.9 KiB, free: 434.4 MiB)
23/06/24 15:00:27 INFO FileFormatWriter: Write Job 24df0e55-195e-42f2-9160-ef4b6316ac03 committed. Elapsed time: 1514 ms.
23/06/24 15:00:27 INFO FileFormatWriter: Finished processing stats for write job 24df0e55-195e-42f2-9160-ef4b6316ac03.
23/06/24 15:00:28 INFO BigQueryClient: Submitted job LoadJobConfiguration{type=LOAD, destinationTable=GenericData{classInfo=[datasetId, projectId, tableId], {datasetId=twitter_kafka_

In [15]:
##### STREAMING DATA PROCESSING #####

# Read the data from kafka
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "latest") \
    .option("header", "true") \
    .load() \
    .selectExpr("CAST(value AS STRING)")
    
# Apply deserialization or further processing if needed
df1 = df.withColumn("parsed_data", from_json("value", schema))

### DATA TYPE CONVERSIONS ####
# Extract and convert the "venue_zipcode" as Integer
# df1 = df1.withColumn("venue_zipcode", col("parsed_data.venue_zipcode").cast(IntegerType()))
# # Extract and convert the coordinates as Double
# df1 = df1.withColumn("venue_longitude", col("parsed_data.venue_longitude").cast(DoubleType()))
# df1 = df1.withColumn("venue_latitude", col("parsed_data.venue_latitude").cast(DoubleType()))
# # Extract and Convert the event_start_date as Date 
# df1 = df1.withColumn("event_start_date", col("parsed_data.event_start_date").cast(DateType()))

df2 = df1.select("parsed_data.*")


df2.printSchema()

path = '/Users/nicburkett/Desktop/spark_output'


# # # Write to a local file
# # file_query = df2.writeStream \
# #     .format("csv") \
# #     .outputMode("append") \
# #     .option("header", "true") \
# #     .option("checkpointLocation", path) \
# #     .trigger(processingTime="10 seconds") \
# #     .start(path)

# # # WRITE TO GCS BUCKET 
# gcs_write = df2.writeStream \
#     .format("csv") \
#     .outputMode("append") \
#     .option("path","gs://kafka-spark-data/raw-spark-data") \
#     .option("checkpointLocation", "gs://kafka-spark-data/spark-metadata") \
#     .trigger(processingTime="10 seconds") \
#     .start() 

# gcs_write.awaitTermination()

# WRITE TO CONSOLE TO LOG 
# console_query = df2.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .trigger(processingTime="10 seconds") \
#     .start() \
#     .awaitTermination()

    # .foreachBatch(write_batch) \

gcs_bigquery_stream = df2.writeStream \
    .format("bigquery") \
    .trigger(processingTime="10 seconds") \
    .option("checkpointLocation", gcs_metadata_folder) \
    .option("temporaryGcsBucket", 'kafka-spark-data') \
    .option("table",'global-maxim-338114.twitter_kafka_pyspark_test.kafka_pyspark') \
    .option("mode", "FAILFAST") \
    .start()

    # .option("failOnDataLoss",'false') \

gcs_bigquery_stream.awaitTermination()

root
 |-- event_name: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- event_url: string (nullable = true)
 |-- venue_name: string (nullable = true)
 |-- venue_id: string (nullable = true)
 |-- venue_zipcode: string (nullable = true)
 |-- venues_timezone: string (nullable = true)
 |-- venue_city: string (nullable = true)
 |-- venue_state_full: string (nullable = true)
 |-- venue_state_short: string (nullable = true)
 |-- venue_country_name: string (nullable = true)
 |-- venue_country_short: string (nullable = true)
 |-- venue_address: string (nullable = true)
 |-- venue_longitude: string (nullable = true)
 |-- venue_latitude: string (nullable = true)
 |-- attraction_name: string (nullable = true)
 |-- attraction_type: string (nullable = true)
 |-- attraction_id: string (nullable = true)
 |-- attraction_url: string (nullable = true)
 |-- attraction_segment_id: string (nullable = true)
 |-- attraction_segment_name: string (nu

23/06/24 15:00:34 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-9756b57f-78f1-47e4-9eaa-8e6b0ed5aa94/' directory.
23/06/24 15:00:34 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor-2, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 0.
23/06/24 15:00:34 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor-2, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor] Seeking to LATEST offset of partition ticketmaster-0
23/06/24 15:00:34 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-executor-2, groupId=spark-kafka-source-71d3a546-2d9e-4f76-a6b9-bc85d9902881--2001384727-execut

23/06/24 15:00:37 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-5f01326b-0233-4789-a924-95976f02eb4e/_temporary/0/_temporary/' directory.
23/06/24 15:00:37 INFO FileOutputCommitter: Saved output of task 'attempt_202306241500357702791542536645745_0002_m_000000_2' to gs://kafka-spark-data/.spark-bigquery-local-1687636163344-5f01326b-0233-4789-a924-95976f02eb4e/_temporary/0/task_202306241500357702791542536645745_0002_m_000000
23/06/24 15:00:37 INFO SparkHadoopMapRedUtil: attempt_202306241500357702791542536645745_0002_m_000000_2: Committed. Elapsed time: 583 ms.
23/06/24 15:00:37 INFO Executor: Finished task 0.0 in stage 2.0 (TID 2). 2579 bytes result sent to driver
23/06/24 15:00:37 INFO TaskSetManager: Finished task 0.0 in stage 2.0 (TID 2) in 1383 ms on nics-mbp.attlocal.net (executor driver) (1/1)
23/06/24 15:00:37 INFO TaskSchedulerImpl: Removed TaskSet 2.0, whose tasks have all completed, from pool 
23/06/24 15:00:

23/06/24 15:00:38 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-5f01326b-0233-4789-a924-95976f02eb4e/_temporary/0/task_202306241500357702791542536645745_0002_m_000000/' directory.
23/06/24 15:00:38 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-5f01326b-0233-4789-a924-95976f02eb4e/' directory.
23/06/24 15:00:38 INFO BlockManagerInfo: Removed broadcast_2_piece0 on nics-mbp.attlocal.net:52028 in memory (size: 95.0 KiB, free: 434.4 MiB)
23/06/24 15:00:38 INFO FileFormatWriter: Write Job 52b6cfab-5929-44a5-b1d6-943b95e2b862 committed. Elapsed time: 1522 ms.
23/06/24 15:00:38 INFO FileFormatWriter: Finished processing stats for write job 52b6cfab-5929-44a5-b1d6-943b95e2b862.
23/06/24 15:00:39 INFO BigQueryClient: Submitted job LoadJobConfiguration{type=LOAD, destinationTable=GenericData{classInfo=[datasetId, projectId, tableId], {datasetId=twitter_kafka_

23/06/24 15:00:50 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor-3, groupId=spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 0.
23/06/24 15:00:50 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor-3, groupId=spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor] Seeking to LATEST offset of partition ticketmaster-0
23/06/24 15:00:50 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor-3, groupId=spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 4839.
23/06/24 15:00:51 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-loca

...

23/06/24 15:19:37 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor-3, groupId=spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 0.
23/06/24 15:19:37 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor-3, groupId=spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor] Seeking to LATEST offset of partition ticketmaster-0
23/06/24 15:19:37 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor-3, groupId=spark-kafka-source-ee43931e-34d0-4e37-a19e-8105f3f9ff7b--2001384727-executor] Resetting offset for partition ticketmaster-0 to offset 5064.
23/06/24 15:19:38 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-loca

23/06/24 15:19:39 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-a56a9574-5b3d-444b-bca9-e09031242c95/_temporary/0/task_202306241519364081636404726572394_0107_m_000000/' directory.
23/06/24 15:19:39 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-a56a9574-5b3d-444b-bca9-e09031242c95/' directory.
23/06/24 15:19:40 INFO FileFormatWriter: Write Job 38873928-196a-423e-90fb-8e2498e046bd committed. Elapsed time: 2008 ms.
23/06/24 15:19:40 INFO FileFormatWriter: Finished processing stats for write job 38873928-196a-423e-90fb-8e2498e046bd.
23/06/24 15:19:40 INFO BigQueryClient: Submitted job LoadJobConfiguration{type=LOAD, destinationTable=GenericData{classInfo=[datasetId, projectId, tableId], {datasetId=twitter_kafka_pyspark_test, projectId=global-maxim-338114, tableId=kafka_pyspark}}, decimalTargetTypes=null, destinationEncryptionConfiguration=null, createD

23/06/24 15:19:48 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-c52b8e18-7ad3-411e-8a21-370717bb009b/_temporary/0/_temporary/' directory.
23/06/24 15:19:48 INFO FileOutputCommitter: Saved output of task 'attempt_202306241519467824561801329708286_0108_m_000000_108' to gs://kafka-spark-data/.spark-bigquery-local-1687636163344-c52b8e18-7ad3-411e-8a21-370717bb009b/_temporary/0/task_202306241519467824561801329708286_0108_m_000000
23/06/24 15:19:48 INFO SparkHadoopMapRedUtil: attempt_202306241519467824561801329708286_0108_m_000000_108: Committed. Elapsed time: 861 ms.
23/06/24 15:19:48 INFO Executor: Finished task 0.0 in stage 108.0 (TID 108). 2579 bytes result sent to driver
23/06/24 15:19:48 INFO TaskSetManager: Finished task 0.0 in stage 108.0 (TID 108) in 1778 ms on nics-mbp.attlocal.net (executor driver) (1/1)
23/06/24 15:19:48 INFO TaskSchedulerImpl: Removed TaskSet 108.0, whose tasks have all completed, from pool 
2

23/06/24 15:19:49 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-c52b8e18-7ad3-411e-8a21-370717bb009b/_temporary/0/task_202306241519467824561801329708286_0108_m_000000/' directory.
23/06/24 15:19:50 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-c52b8e18-7ad3-411e-8a21-370717bb009b/' directory.
23/06/24 15:19:50 INFO BlockManagerInfo: Removed broadcast_108_piece0 on nics-mbp.attlocal.net:52028 in memory (size: 94.9 KiB, free: 434.4 MiB)
23/06/24 15:19:50 INFO FileFormatWriter: Write Job c0859031-079f-46f1-8a87-fccc164bcf41 committed. Elapsed time: 2200 ms.
23/06/24 15:19:50 INFO FileFormatWriter: Finished processing stats for write job c0859031-079f-46f1-8a87-fccc164bcf41.
23/06/24 15:19:51 INFO BigQueryClient: Submitted job LoadJobConfiguration{type=LOAD, destinationTable=GenericData{classInfo=[datasetId, projectId, tableId], {datasetId=twitter_kafk

23/06/24 15:19:58 INFO GoogleCloudStorageFileSystem: Successfully repaired 'gs://kafka-spark-data/.spark-bigquery-local-1687636163344-2feeea02-d93b-44b6-bb32-a0abd9b5baea/_temporary/0/_temporary/' directory.
23/06/24 15:19:58 INFO FileOutputCommitter: Saved output of task 'attempt_202306241519568872264779257972892_0109_m_000000_109' to gs://kafka-spark-data/.spark-bigquery-local-1687636163344-2feeea02-d93b-44b6-bb32-a0abd9b5baea/_temporary/0/task_202306241519568872264779257972892_0109_m_000000
23/06/24 15:19:58 INFO SparkHadoopMapRedUtil: attempt_202306241519568872264779257972892_0109_m_000000_109: Committed. Elapsed time: 948 ms.
23/06/24 15:19:58 INFO Executor: Finished task 0.0 in stage 109.0 (TID 109). 2579 bytes result sent to driver
23/06/24 15:19:58 INFO TaskSetManager: Finished task 0.0 in stage 109.0 (TID 109) in 1857 ms on nics-mbp.attlocal.net (executor driver) (1/1)
23/06/24 15:19:58 INFO TaskSchedulerImpl: Removed TaskSet 109.0, whose tasks have all completed, from pool 
2

In [42]:
# WRITE TO CONSOLE TO LOG 
console_query = df2.writeStream \
    .format("console") \
    .outputMode("append") \
    .trigger(processingTime="10 seconds") \
    .start() \
    .awaitTermination()

    # .foreachBatch(write_batch) \

23/06/22 20:39:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/temporary-7718572b-fe8e-461e-9e94-f12a490f995c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/06/22 20:39:55 INFO ResolveWriteToStream: Checkpoint root /private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/temporary-7718572b-fe8e-461e-9e94-f12a490f995c resolved to file:/private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/temporary-7718572b-fe8e-461e-9e94-f12a490f995c.
23/06/22 20:39:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/06/22 20:39:55 INFO CheckpointFileManager: Writing atomically to file:/private/var/folders/gr/l9vf1gtd2_s_3tz5n15mgkn80000gn/T/te

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nicburkett/opt/anaconda3/envs/spark/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

23/06/22 20:40:20 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0-33, groupId=spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0] Seeking to LATEST offset of partition ticketmaster-0
23/06/22 20:40:20 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0-33, groupId=spark-kafka-source-2b8cbc25-4bf7-465c-b61f-6e26ba34a2b9--2109907342-driver-0] Resetting offset for partition ticketmaster-0 to offset 1393.
23/06/22 20:40:20 INFO MicroBatchExecution: Streaming query made progress: {
  "id" : "5ff3f0d9-8649-4084-a0de-9c984d47f474",
  "runId" : "061d6392-20e9-47dc-a164-da2a57f51557",
  "name" : null,
  "timestamp" : "2023-06-23T01:40:20.005Z",
  "batchId" : 1,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "latestOffset" : 10,
    "triggerExecution" : 

In [23]:
spark.stop()

23/06/19 19:50:27 INFO FetchSessionHandler: [Consumer clientId=consumer-spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor-9, groupId=spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor] Error sending fetch request (sessionId=INVALID, epoch=INITIAL) to node 1001:
org.apache.kafka.common.errors.TimeoutException: Failed to send request after 30000 ms.
23/06/19 19:50:42 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor-9, groupId=spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor] Connection to node 1001 (host.docker.internal/143.244.220.150:9092) could not be established. Broker may not be available.
23/06/19 19:50:42 INFO FetchSessionHandler: [Consumer clientId=consumer-spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor-9, groupId=spark-kafka-source-d2941ca6-4ad1-48ba-bda3-65af4267d63f--2001384727-executor] Erro

In [ ]:

## BATCH DATA PROCESSING 

df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic_name) \
  .load()\
  .selectExpr("CAST(value AS STRING)")

# Apply deserialization or further processing if needed
df1 = df.withColumn("parsed_data", from_json("value", schema))
## Select the data from the parsed_data column
df2 = df1.select("parsed_data.*")

gcs_metadata_folder = "gs://kafka-spark-data/spark-metadata"
gcs_data_folder = "gs://kafka-spark-data/raw-spark-data"

print(df2.schema)

## WRITE TO LOCAL STORAGE
# gcs_write = df2.write \
#   .format("csv") \
#   .option("checkpointLocation", "/Users/nicburkett/Desktop/spark_output") \
#   .option("path","/Users/nicburkett/Desktop/spark_output") \
#   .mode("overwrite")\
#   .save()

##WRITE TO GCS BUCKET
# gcs_write_newfolder = df2.write \
#   .format("parquet") \
#   .option("checkpointLocation", gcs_metadata_folder) \
#   .option("path",gcs_data_folder) \
#   .mode("overwrite")\
#   .save()


dataset = 'global-maxim-338114.twitter_kafka_pyspark_test'
table = 'twitter_kafka_pyspark_test'

# Write the DataFrame to BigQuery
 ## this is the bucket where the data is stored temporarily
df2.write \
    .format("bigquery") \
    .mode("overwrite") \
    .option("checkpointLocation", gcs_metadata_folder) \
    .option("temporaryGcsBucket", 'kafka-spark-data') \
    .option("table",'global-maxim-338114.twitter_kafka_pyspark_test.kafka_pyspark') \
    .option("mode", "FAILFAST") \
    .save()


In [ ]:
import pandas as pd
# Function to save the data to GCS with a custom filename
def save_to_gcs(batch_df, batch_id):
    # Convert the batch dataframe to a pandas dataframe
    pandas_df = batch_df.toPandas()

    # Get the values of the desired columns from the first row
    column1_value = pandas_df.loc[0, "column1"]
    column2_value = pandas_df.loc[0, "column2"]

    # Get the current time
    current_time = pd.Timestamp.now()

    # Generate the custom filename
    filename = f"file_{column1_value}_{column2_value}_{current_time}.parquet"

    # Save the dataframe to GCS with the custom filename
    pandas_df.to_parquet(f"gs://{bucket_name}/{path}/{filename}", index=False)


In [ ]:
# DATA TYPES THOUGHOUT KAFKA SERVER
import json
import pandas as pd
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, udf
import findspark

findspark.init()
topic_name = 'twitter'
# Config
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("TwitterSentimentAnalysis") \
    .getOrCreate()

prod = {'user_id': 19, 'recipient_id': 57, 'message': 'YbfyRHyWgjuGlzOiudEcVMLJNzqUPDvV'}
print(type(prod))
##convert dictionary to json string (BYTES)
serialized_prod = json.dumps(prod).encode('utf-8')

print(f'The producer dtype is {type(serialized_prod)} and output is {(serialized_prod)}')

## turn from string/bytes into a dictionary again
deserializer_function = lambda x: json.loads(x.decode('utf-8'))
deserialized_cons = deserializer_function(serialized_prod)

print(f'The producer dtype is {type(deserialized_cons)} and output is {(deserialized_cons)}')

### PARSING THE JSON COMING OUT 
user_id = deserialized_cons.get('user_id')
recipient_id = deserialized_cons.get('recipient_id')
message = deserialized_cons.get('message')
output_parsed = print(f'UserID: {user_id}, RecipientID: {recipient_id}, Message:{message}')


df_pandas = pd.DataFrame([deserialized_cons])
df_pandas


# df_spark = spark.createDataFrame(df_pandas)
# df_spark.show()

# Create a spark schema/column headers
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("recipient_id", IntegerType(), True),
    StructField("message", StringType(), True)
])

# Create DataFrame from a single row
# data = [(user_id, recipient_id, message)]
df_spark = spark.createDataFrame(df_pandas,schema)
df_spark.show()
# df.write.csv('/path/to/output.csv', header=True, mode='overwrite')
